In [ ]:
# pip install imgaug

In [1]:
import random
import numpy as np
import os
import shutil
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from PIL import Image
from glob import glob
from PIL import Image, ImageDraw, ImageFont, ImageOps
from tqdm import tqdm
import PIL.ImageOps
# imgaug의 시드 고정
ia.seed(42)
# directory로 이동
# %cd "./yolov5/data_crop"

In [4]:
def route(t):
    file_path = os.path.join("./images", t)
    pth = []
    for f in os.listdir(file_path):
        pth.append(os.path.join(file_path, f))
    return pth
def annot_route(t):
    file_path = os.path.join("./labels", t)
    pth = []
    for f in os.listdir(file_path):
        pth.append(os.path.join(file_path, f))
    return pth
pth_tr = route("train")
annot_tr = annot_route("train")

In [6]:
annot_tr

['./labels\\train\\PartB_01401.txt',
 './labels\\train\\PartB_01402.txt',
 './labels\\train\\PartB_01403.txt',
 './labels\\train\\PartB_01404.txt',
 './labels\\train\\PartB_01405.txt',
 './labels\\train\\PartB_01406.txt',
 './labels\\train\\PartB_01407.txt',
 './labels\\train\\PartB_01408.txt',
 './labels\\train\\PartB_01409.txt',
 './labels\\train\\PartB_01410.txt',
 './labels\\train\\PartB_01411.txt',
 './labels\\train\\PartB_01412.txt',
 './labels\\train\\PartB_01413.txt',
 './labels\\train\\PartB_01414.txt',
 './labels\\train\\PartB_01415.txt',
 './labels\\train\\PartB_01417.txt',
 './labels\\train\\PartB_01418.txt',
 './labels\\train\\PartB_01419.txt',
 './labels\\train\\PartB_01420.txt',
 './labels\\train\\PartB_01421.txt',
 './labels\\train\\PartB_01422.txt',
 './labels\\train\\PartB_01423.txt',
 './labels\\train\\PartB_01424.txt',
 './labels\\train\\PartB_01425.txt',
 './labels\\train\\PartB_01426.txt',
 './labels\\train\\PartB_01427.txt',
 './labels\\train\\PartB_01428.txt',
 

### Data Augmentation 대상
- class imbalance를 해결하기 위해서 Augmentation을 진행한다.
- annotation 中 3번 클래스가 상당 수를 차지하는 case에 대해서 augmentation을 진행한다

In [ ]:
# class 파악
# 1번 클래스가 있는 경우는 무조건 다 augmentation의 대상이 된다.
# 각 사진 별로 3장씩 augmentation을 진행한다.
def classAnalysis(pth):
    l = []
    cnt = 0
    with open(pth, "r") as f:
        annots = f.readlines()
        for line in annots:
            if line[0] == '3':
                cnt += 1
            l.append(line[0])
    return cnt / len(l)
annot_pth = []
for idx in range(len(annot_tr)):
    ratio = classAnalysis(annot_tr[idx])
    if ratio > 0:
        annot_pth.append([annot_tr[idx], idx])

In [ ]:
image_pth = []
for pth in annot_pth:
    imgs = pth_tr[pth[1]]
    image_pth.append(imgs)

In [ ]:
# annotation 정보의 class 숫자를 파악.
ann = annot_tr
belt, nobelt, hard, nohard, helmet, nohelmet = 0, 0, 0, 0, 0, 0
for i in ann:
    with open(i, "r") as f:
        data = f.readlines()
    for cl in data:
        if cl[0] == '0':
            belt += 1
        if cl[0] == "1":
            nobelt += 1
        if cl[0] == "2":
            hard += 1
        if cl[0] == "3":
            nohard += 1
        if cl[0] == "4":
            helmet += 1
        if cl[0] == "5":
            nohelmet += 1
print("the number of hard class: {}, the number of no-hard class: {}, the number of helmet class: {}, the number of no-helmet class: {}, the number of belt class: {},the number of no-belt class: {}".format(helmet, head, helmet, nohelmet, belt, nobelt))

### 총 3가지의 Augmentation을 구현
1. 180°회전(Rotation)
2. Noise 추가
3. 객체를 기준으로 한 Crop

In [ ]:
"""
Noise를 추가하는 함수
최종적으로는 사용하지 않은 부분입니다.
"""

def make_noise(std, img):
    height, width, ch = img.shape
    # 각 channel에 적용
    # 하나 더 추가
    # 빈 배열 형성
    img_noise = np.zeros((height, width, ch), dtype = np.float64)
    for i in range(height):
        for j in range(width):
            for k in range(ch):
                # gaussian distribution을 따르는 random 숫자를 넣는다
                mk_noise = np.random.normal()
                set_noise = std * mk_noise
                noise_input = img[i][j][k] + set_noise
                if noise_input > 255:
                    noise_input = 255
                if noise_input < 0:
                    noise_input = 255
                img_noise[i][j][k] = noise_input
    return img_noise

In [16]:
"""
객체를 기준으로 Crop하는 함수
Bounding box 중 x_center, y_center의 전체 이미지 상에서의 상대적인 위치에 대한 지정에 신경 썼습니다.
"""
def make_crop(img_annot, image):
    with open(img_annot, "r") as f:
        data = f.readlines()
        # cv2는 height, width, channel 순이다.
        img = image
        height, width = img.shape[0], img.shape[1]
        # 각 line에서 xmin, xmax, ymin, ymax를 뽑아낸다.
        xmin = []
        ymin = []
        xmax = []
        ymax = []
        xcenter = []
        ycenter = []
        new_xcenter = []
        new_ycenter = []
        WIDTH = []
        w_l = []
        HEIGHT = []
        h_l = []
        class_no = []
        for line in data:
            info = line.split()
            x_center, y_center, w, h = float(info[1]), float(info[2]), float(info[3]), float(info[4])
            class_no.append(int(line[0]))
            xcenter.append(int(x_center * width))
            ycenter.append(int(y_center * height))
            xmin.append(int((x_center - w/2) * width))
            ymin.append(int((y_center - h/2) * height))
            xmax.append(int((x_center + w/2) * width))
            ymax.append(int((y_center + h/2) * height))
            WIDTH.append(int(w * width))
            HEIGHT.append(int(h * height))
        xmin, ymin, xmax, ymax = min(xmin), min(ymin), max(xmax), max(ymax)
        # crop width
        crop_width = xmax - xmin
        crop_height = ymax - ymin
        for wid in WIDTH:
            w_l.append(wid/crop_width)
        for hei in HEIGHT:
            h_l.append(hei/crop_height)
        for x in xcenter:
            new_xcenter.append((x - xmin)/crop_width)
        for y in ycenter:
            new_ycenter.append((y - ymin)/crop_height)
    return xmin, ymin, xmax, ymax, xcenter, ycenter, new_xcenter, new_ycenter, w_l, h_l, class_no

In [17]:
# 1. 사진을 rotation
# 2. CenterCrop을 진행
# num값의 조정을 통해서 선택적으로 증강을 진행할 수 있다.
def data_augmentation(num, img, annot, n):
    image = cv2.imread(img)
    img_pth = img
    img_annot = annot
    """
    180°회전하는 부분(num == 0)
    """
    if num == 0:
        # annotation file을 읽는다.
        f = open(img_annot)
        bbx = f.readlines()
        # normalize된 것을 복구하기 위한 작업
        d_w = image.shape[1] # 폭
        d_h = image.shape[0] # 높이
        # 하나씩 읽어온다.
        class_no = []
        for i in range(len(bbx)):
            bbxs = bbx[i].split(" ")
            for j in range(len(bbxs)):
                # 이는 class에 해당한다.
                if j == 0:
                    bbxs[j] = int(bbxs[j])
                    class_no.append(int(bbxs[j]))
                # str 형태를 float으로 바꿔온다
                else:
                    x = float(bbxs[j])
                    bbxs[j] = x
            # 대체하는 작업
            bbx[i] = bbxs
        # 작업해야할 bounding box
        bbx_new = []
        for i in range(len(bbx)):
            # 원래의 x_center
            x_center = bbx[i][1] * d_w
            # 원래의 y_center
            y_center = bbx[i][2] * d_h
            # 원래의 폭
            w = bbx[i][3] * d_w
            # 원래의 높이
            h = bbx[i][4] * d_h
            # x1, x2는 각각 boundingbox의 x좌표의 양 끝단
            # y1, y2는 각각 boundingbox의 y좌표의 양 끝단
            x1 = int(x_center - w/2)
            y1 = int(y_center - h/2)
            x2 = int(x_center + w/2)
            y2 = int(y_center + h/2)
            bbx_new.append([x1, y1, x2, y2])
        ia_bbxs = []
        for b in bbx_new:
            ia_bbxs.append(BoundingBox(x1 = b[0], y1 = b[1], x2 = b[2], y2 = b[3]))
        bound_box = BoundingBoxesOnImage(ia_bbxs, shape = image.shape)
        # 180도 회전을 다음과 같이 진행한다.
        seq = iaa.Sequential([iaa.Affine(rotate = 180)])
        # 180도 회전 적용
        img_aug, bbx_aug = seq(image = image, bounding_boxes = bound_box)
        # 이미지를 각 파일에 저장
        storage_dir = os.path.join(img_pth[:14], "rotation_"+str(n)+".png")
        cv2.imwrite(storage_dir, img_aug)
        # 정규화를 시켜서 저장한다.
        annot_file_path = os.path.join(img_annot[:14], "rotation_"+str(n)+".txt")
        with open(annot_file_path, "w") as f:
            for a in range(len(bbx_aug)):
                class_id = class_no[a]
                # opencv의 경우는 높이, 폭, 채널 순서
                dx = 1/int(image.shape[1])
                dy = 1/int(image.shape[0])
                x1 = bbx_aug[a][0][0]
                x2 = bbx_aug[a][1][0]
                y1 = bbx_aug[a][0][1]
                y2 = bbx_aug[a][1][1]
                x_center = (x1+x2)/2 * dx
                y_center = (y1+y2)/2 * dy
                w = (x2-x1) * dx
                h = (y2-y1) * dy
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")
    # Noise 추가
    # 기본적으로 noise로는 Gaussian noise를 추가한다.
    # if num == 1:
    #     noise_img = make_noise(12, image)
    #     storage_dir = os.path.join(img_pth[:14], "noise_"+str(n)+".png")
    #     # 이미지 저장
    #     cv2.imwrite(storage_dir, noise_img)
    #     # annotation 저장
    #     new_annot_file_path = os.path.join(img_annot[:14], "noise_"+str(n)+".txt")
    #     original_annot_path = img_annot
    #     shutil.copyfile(original_annot_path, new_annot_file_path)
    # crop 진행.
    if num == 1:
        # Center 값만 변형 시킨다.
        # crop을 한다고 해서 Width와 height가 변하는 것은 아니다
        # 시작 기준점이 바뀐다.
        mk_annot = make_crop(img_annot, image)
        xmin, xmax, ymin, ymax = mk_annot[0], mk_annot[2], mk_annot[1], mk_annot[3]
        cropped_image = image[ymin:ymax, xmin:xmax]
        storage_dir = os.path.join(img_pth[:14], "cropped_"+str(n)+".png")
        cv2.imwrite(storage_dir, cropped_image)
        # annotation을 적어야 한다.
        annot_file_path = os.path.join(img_annot[:14], "cropped_"+str(n)+".txt")
        # new_xcenter, new_ycenter
        Xcenter = mk_annot[6]
        Ycenter = mk_annot[7]
        width_list = mk_annot[8]
        height_list = mk_annot[9]
        class_no = mk_annot[10]
        with open(annot_file_path, "w") as f:
            for idx in range(len(class_no)):
                class_id = class_no[idx]
                x_center = Xcenter[idx]
                y_center = Ycenter[idx]
                w = width_list[idx]
                h = height_list[idx]
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")

In [18]:
for n in range(len(annot_tr)):
    annot = annot_tr[n]
    imgs = pth_tr[n]
    # num이 0, 1 순서대로 rotation, Gaussian Noise, Crop
    for num in range(2):
        data_augmentation(num, imgs, annot, n)

In [ ]:
dir_main = "./data_crop"
filenames_image = glob(f"{dir_main}/images/train/*.png")[-5:]
filenames_label = [filename.replace('images', 'labels').replace('png', 'txt') for filename in filenames_image]
classes = ["hard", "no hard", "helmet", "no helmet", "belt", "no belt"]

color = []
# color 생성하는 부분
for _ in range(6):
    c = list(np.random.choice(range(256), size=3)) + [255]
    c = tuple(c)
    color.append(c)


# annotation 정보가 제대로 저장됐는지 확인하기 위한 bounding box를 시각화하는 함수
def draw_bbox(draw, bbox, label, color=(0, 255, 0, 255), confs=None, size=15):
    font = ImageFont.truetype("arial.ttf", size=size)
    draw.rectangle(bbox, outline=color, width =3)
    def set_alpha(color, value):
        background = list(color)
        background[3] = value
        return tuple(background)
    background = set_alpha(color, 50)
    draw.rectangle(bbox, outline=color, fill=background, width =3)
    background = set_alpha(color, 150)
    text = f"{label}" + ("" if confs==None else f":{conf:0.4}")
    text_bbox = bbox[0], bbox[1], bbox[0]+len(text)*10, bbox[1]+25
    draw.rectangle(text_bbox, outline=color, fill=background, width =3)
    draw.text((bbox[0]+5, bbox[1]+5), text, (0,0,0), font=font)

cnt = 3
for filename_image, filename_label in tqdm(zip(filenames_image, filenames_label)):
    img = Image.open(filename_image)
    img = img.resize((640, 640))
    width, height = img.size
    draw = ImageDraw.Draw(img, 'RGBA')
    with open(filename_label, 'r') as f:
        labels = f.readlines()
        # strip: 개행 문자 제거.
        labels = list(map(lambda s: s.strip().split(), labels))
    # bounding box를 그리는 부분
    for label in labels:
        # class는 0부터 시작!
        cls = int(label[0])
        x, y, w, h = map(float, label[1:])
        x1, x2 = width * (x-w/2), width * (x+w/2)
        y1, y2 = height * (y-h/2), height * (y+h/2)
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        draw_bbox(draw, bbox=(x1, y1, x2, y2), label=classes[cls], color=color[cls], size=15)
    img.
    img.show()

    cnt -= 1
    if cnt ==0:
        break